In [28]:
import os
import argparse
import SimpleITK as sitk
from collections import defaultdict
import json
import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument('-rp', '--results_path', type=str)
parser.add_argument('-tdp', '--test_data_path', type=str)

args = argparse.Namespace(
    test_data_path = '/home/t722s/Desktop/Datasets/BratsMini/',
    results_path = '/home/t722s/Desktop/Sam-Med3DTest/BratsMini'
)

def compute_dice(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.
    Returns:
    the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2*volume_intersect / volume_sum


results_path = args.results_path
labels_dir = os.path.join(args.test_data_path, 'labelsTs')
with open(os.path.join(args.test_data_path, 'dataset.json'), 'r') as f:
    dataset_metadata = json.load(f)

labels_dict = dataset_metadata['labels']

eval_res = defaultdict(dict)

for label_name in os.listdir(labels_dir):
    label = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(labels_dir, label_name)))
    base_name = label_name.replace('.nii.gz','')
    
    for organ, label_num in labels_dict.items():
        label_num = int(label_num) # since it might be a string due to JSON
        label_binary = np.where(label == label_num, 1, np.zeros_like(label))
        organ_dir = os.path.join(args.results_path, organ)        

        seg_names = [file for file in os.listdir(organ_dir) if file.startswith(base_name + f'_pred_{organ}_trans')]

        organ_res = dict()
        for seg_name in seg_names:
            
            seg = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(organ_dir, seg_name)))
            seg_number = seg_name.removeprefix(base_name + f'_pred_{organ}_trans').removesuffix('.nii.gz') # Get segmentation number, ie number of clicks.
            dice = compute_dice(label, seg)
            organ_res[int(seg_number)+1] = dice
            eval_res[base_name][organ] = organ_res
            
with open(os.path.join(results_path, 'evaluation_dice.json'), 'w') as f:
    json.dump(eval_res, f, indent = 4)

looking at f/home/t722s/Desktop/Datasets/BratsMini/labelsTs/BraTS2021_01646.nii.gz
labels: [0 1], looking in /home/t722s/Desktop/Sam-Med3DTest/BratsMini/Brain Tumor
 Found files: ['BraTS2021_01646_pred_Brain Tumor_trans0.nii.gz', 'BraTS2021_01646_pred_Brain Tumor_trans1.nii.gz', 'BraTS2021_01646_pred_Brain Tumor_trans2.nii.gz', 'BraTS2021_01646_pred_Brain Tumor_trans3.nii.gz', 'BraTS2021_01646_pred_Brain Tumor_trans4.nii.gz'], pattern: BraTS2021_01646Brain Tumor_pred_trans


In [21]:
base_name

'BraTS2021_01646'

In [27]:
eval_res

defaultdict(dict,
            {'BraTS2021_01646': {'Brain Tumor': {1: 0.7663380680923435,
               2: 0.6933359543935522,
               3: 0.7110890394952092,
               4: 0.7182991202346041,
               5: 0.7024892502998216}}})